Материалы:

- https://www.youtube.com/watch?v=maMReTuUOWA
http://uneex.org/LecturesCMC/PythonIntro2020/14_Async

- https://www.youtube.com/watch?v=5SyA3lsO_hQ

- Fluent Python 2021 pre-release, Luciano Ramalho главы 19-21

- https://docs.python.org/3/library/asyncio.html

# Speeding up

Перед обсуждением более сложных концептов, вспомним необходимые термины и зайдем чуть издалека

## Generators and coroutines

### Gen. functions with yield

**Функция-генератор**

In [70]:
def gen_fun():
    yield 42
    yield 43
    yield 44

In [71]:
for v in gen_fun():
    print(v)

42
43
44


**Объект-генератор**

In [72]:
obj_fun = gen_fun()

In [73]:
next(obj_fun)

42

In [74]:
obj_fun.__next__()

43

In [76]:
obj_fun.send(None)

44

In [77]:
next(obj_fun)

StopIteration: 

Можно делать стримы. Например, нагенерировать последовательно Фибоначчи в виде генератора, постепенно читать из больших файлов и т.д. 

**Можно делать и return**

In [118]:
def gen_fun():
    yield 42
    yield 43
    yield 44
    return 'string'

In [119]:
obj_fun = gen_fun()

In [120]:
next(obj_fun)

42

In [121]:
next(obj_fun)

43

In [122]:
next(obj_fun)

44

In [123]:
try:
    next(obj_fun)
except StopIteration as e:
    print(e)
    print(e.with_traceback)
    print(e.value)

string
<built-in method with_traceback of StopIteration object at 0x7f37d15b6dc0>
string


### Coroutines

In [62]:
from collections.abc import Generator

In [66]:
def averager() -> Generator:
    total = 0.
    count = 0
    avg = 0.
    while True:
        curr = yield avg
        total += curr
        count += 1
        avg = total / count

In [67]:
coro_avg = averager()
next(coro_avg)

0.0

In [68]:
coro_avg.send(10)

10.0

In [69]:
coro_avg.send(15)

12.5

Очень похоже на рассмотренный ранее на курсе пример с замыканием. Корутина хранит состояние.

В Python корутины -- это по сути те же генераторы, только мы еще помимо считывания данных, их туда дописываем

In [78]:
coro_avg.send(20)

15.0

In [79]:
coro_avg.close()

In [80]:
coro_avg.send(20)

StopIteration: 

Неформально, **корутина** (или сопрограмма в старой терминологии) -- это такая функция, в которую можно входить много раз и каждый раз попадать именно в то место, из которого в прошлый раз выходил. А также при очеденом входе в такую функцию можно добавлять новые данные. После частичного выхода из сопрограммы, ее код перестает выполняться.

Также обратим внимание на конструкцию **yield from**

In [90]:
from time import sleep
def secondary():
    yield "SEC 1"
    yield "SEC 2"

def primary(name="Prim"):
    while True:
        yield from secondary()
        yield name

In [91]:
core = primary()

n = 12
for i in range(n):
    sleep(1)
    print(f'{i} {next(core)}')

0 SEC 1
1 SEC 2
2 Prim
3 SEC 1
4 SEC 2
5 Prim
6 SEC 1
7 SEC 2
8 Prim
9 SEC 1
10 SEC 2
11 Prim


Что если сделать два таких primary исполнителя?

In [95]:
from random import random

def secondary(src_name):
    yield f'SEC 1 from {src_name}'
    yield f'SEC 2 from {src_name}'

def primary(name='prim'):
    for i in range(1000):
        yield from secondary(name)
        yield f'{name}-{i}'


core_one = primary('first prim')
core_two = primary('second prim')

n = 30
threshold = 0.8

for i in range(n):
    sleep(1)
    if random() > threshold:
        res = next(core_one)
    else:
        res = next(core_two)
    print(f'{i} {res}')

0 SEC 1 from second prim
1 SEC 2 from second prim
2 second prim-0
3 SEC 1 from second prim
4 SEC 2 from second prim
5 second prim-1
6 SEC 1 from second prim
7 SEC 2 from second prim
8 second prim-2
9 SEC 1 from second prim
10 SEC 1 from first prim
11 SEC 2 from first prim
12 SEC 2 from second prim
13 first prim-0
14 second prim-3
15 SEC 1 from second prim
16 SEC 2 from second prim
17 second prim-4
18 SEC 1 from second prim
19 SEC 2 from second prim
20 second prim-5
21 SEC 1 from second prim
22 SEC 2 from second prim
23 second prim-6
24 SEC 1 from second prim
25 SEC 2 from second prim
26 second prim-7
27 SEC 1 from second prim
28 SEC 1 from first prim
29 SEC 2 from first prim


А что если было возвращаемое значение?

In [126]:
def secondary(src_name, i):
    yield f'SEC 1 from {src_name}'
    yield f'SEC 2 from {src_name}'
    return src_name * i
    
def primary(name='prim'):
    for i in range(1000):
        res = yield from secondary(name, i)
        print(f'secondary-{i}: {res}')
        
core = primary()

n = 12
for i in range(n):
    sleep(1)
    print(f'{i} {next(core)}')

0 SEC 1 from prim
1 SEC 2 from prim
secondary-0: 
2 SEC 1 from prim
3 SEC 2 from prim
secondary-1: prim
4 SEC 1 from prim
5 SEC 2 from prim
secondary-2: primprim
6 SEC 1 from prim
7 SEC 2 from prim
secondary-3: primprimprim
8 SEC 1 from prim
9 SEC 2 from prim
secondary-4: primprimprimprim
10 SEC 1 from prim
11 SEC 2 from prim


Также генераторы могут друг другу пересылать данные через **.send**

In [130]:
def value_computer(val):
    x = yield f'computer: get val x '
    y = yield f'computer: get val y '
    return x ** 2 + y

def value_provider():
    res_one = yield from value_computer(1)
    res_two = yield from value_computer(2)
    
    return res_one + res_two + 100

command = value_provider()
req = next(command)

for i in range(4):
    data = eval(input(req))
    req = command.send(data)  # send в computer, у которого больше нет yield

computer: get val x 1
computer: get val y 2
computer: get val x 3
computer: get val y 4


StopIteration: 116

Теперь значения будем не руками передавать, а делать это из цикла. Например, если у нас сгенерирован int, будем засылать рандомную цифру, str -- букву

Делаем асинхронность своими руками

In [142]:
import random
from collections import deque


def computer():
    '''
    умножаем строчку на число
    '''
    return (yield int) * (yield str)

def repeater(n):
    '''
    склеиваем n строчек
    '''
    res = ''
    for i in range(n):
        res += yield from computer()
    
    return res

def runner(*commands):
    curr_queue = deque((command, None) for command in reversed(commands))
    res = []
    
    while curr_queue:
        cmd, req_type = curr_queue.pop()
        try:
            if req_type is str:
                req_type = cmd.send(random.choice('ABCDRTYUWETDGH'))
            elif req_type is int:
                req_type = cmd.send(random.randint(1, 7))
            elif req_type is None:
                req_type = next(cmd)
            else:
                raise ValueError(req_type)
        except StopIteration as e:
            res.append(e.value)
            cmd.close()
            # когда получили StopIteration, генератор еще существует
            # закрытие как защита от непредвиденного поведения
        else:
            curr_queue.appendleft((cmd, req_type))
    return res

In [141]:
print(runner(repeater(10), repeater(2), repeater(6)))

['CCCT', 'AAEEEBEEEEEEECCCCCGGGG', 'DDDDDDDAAAGGGGGAAAAACCCCCCCTTTTEEEEDDDDDDWWWHH']


Код выполняется в произвольном порядке (но все еще контролируемым программистом!)

А теперь следите за руками

In [145]:
import asyncio
import types

In [150]:
import random
from collections import deque

@types.coroutine
def computer():
    '''
    умножаем строчку на число
    '''
    return (yield int) * (yield str)

async def repeater(n):
    '''
    склеиваем n строчек
    '''
    res = ''
    for i in range(n):
        res += await computer()
    
    return res

def runner(*commands):
    curr_queue = deque((command, None) for command in reversed(commands))
    res = []
    
    while curr_queue:
        cmd, req_type = curr_queue.pop()
        try:
            if req_type is str:
                req_type = cmd.send(random.choice('ABCDRTYUWETDGH'))
            elif req_type is int:
                req_type = cmd.send(random.randint(1, 7))
            elif req_type is None:
                req_type = cmd.send(None)
            else:
                raise ValueError(req_type)
        except StopIteration as e:
            res.append(e.value)
            cmd.close()
            # когда получили StopIteration, генератор еще существует
            # закрытие как защита от непредвиденного поведения
        else:
            curr_queue.appendleft((cmd, req_type))
    return res

In [151]:
print(runner(repeater(10), repeater(2), repeater(6)))

['HHHEEEEE', 'GGGGGTBBBBBBBEEEEEEECCCCCCCT', 'TTTTTTTTTTTTACGGGRRRTTTTTTTGGGGGGGGHHHHHH']


Мы сейчас практически из кода на чистом питоне с детерминированным выполнением фрагментов кода, перешли к использованию **asyncio**!

## Concurrency, processes, threads

**concurrency**

The ability to handle multiple pending tasks, making progress one at a time or in parallel (if possible) so that each of them eventually succeeds or fails. A single-core CPU is capable of concurrency if it runs an OS scheduler that interleaves the execution of the pending tasks. Also known as multitasking

**parallelism**

The ability to execute multiple computations at the same time. This requires a multi-core CPU, a GPU, or multiple computers in cluster.

**process**

An instance of a computer program while it is running, using memory and a slice of the CPU time. Modern operating systems are able to manage many processes concurrently, with each process isolated in its own private memory space. Processes communicate via pipes, sockets, or memory mapped files—all of which can only carry raw bytes, not live Python objects. A process can spawn sub-processes, each called achild process. These are also isolated from each other and from the parent.

Каждый инстант Python интерпретатора -- это процесс. Можно использовать модуль *multiprocessing*

**thread**

An execution path within a single process. When a process starts, it uses a single thread: the main thread. Using operating system APIs, a process can create more threads that operate concurrently thanks to the operating system scheduler. Threads share the memory space of the process, which holds live Python objects. This allows easy communication between threads, but can also lead to corrupted data when more than one thread updates the same object concurrently.

Python (CPython) использует один тред. Можно добавлять новые с помощью *threading*

**contention**

Dispute over a limited asset. Resource contention happens when multiple processes or threads try to access a shared resource—such as a lock or storage. There’s also CPU contention, when compute-intensive processes or threads must wait for their share of CPU time.

**lock**

An object that threads can use to coordinate and synchronize their actions and avoid corrupting data. While updating a shared data structure, a thread should hold an associated lock. This makes other well-behaved threads wait until the lock is released before accessing the same data structure. The simplest type of lock is also known as a mutex (for mutual exclusion).

В Python есть такая вещь как GIL (Global Interpreter Lock). В один момент времени только один тред может держать данный лок

Интерпретатор каждые несколько мс делает паузы, освобождая данный лок и позволяя другому ожидающему треду продолжить работу

Мы не управляем GIL явно

GIL освобождается во время функций, производящих disk и network I/O + time.sleep(). Библиотеки, написанные на плюсях, сях, тоже могут такое проворачивать и улучшать производительность (e.g. numpy)

---

Как могут сказаться отличия работы с памятью в программах на треды и процессы, если добавить туда относительно тяжелый список?

Что будет, если в серии программ **loading_.** заменить медленную функцию на такую?

In [153]:
def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

In [2]:
! pip install requests, aiohttp

     |████████████████████████████████| 62 kB 659 kB/s eta 0:00:01
     |████████████████████████████████| 61 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████| 138 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 11.8 MB/s eta 0:00:01
